# Применение Computer Vision в задаче медицинской диагностики

---



## Датасет для обучения сети был взят с kaggle, он содержит рентген-снимки детей в возрасте до 5 лет. 

## Для обучения нейросети (а если быть более точным 5-и экземпляров) я использовал сервис Google Colab. Здесь же находятся лишь модель и 5 обученных весов. Процесс обучения показывать я не буду, так как он занимает больше 30-и минут на одну модель. Снимок проходит сначала через 5 моделей, их предсказания объединяются в единий "слепок", который затем отправляется полносвязой сети, обученной для анализа уже предсказанных данных. Полносвязанная сеть на основе "слепка" уже делает свои предсказания и вывод. Точность всей конструкции составляет 93%.

### Здесь находятся все необходимые импорты.

In [0]:
import os
import numpy as np
import cv2
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

### Этот фрагмент — загрузка в оперативную память данных для обучения. Я перемешиваю данные, так как изначально они идут в последовательном порядке (сначала обычные снимки, затем с пневмонией)

In [0]:
path = os.getcwd() + '\data\chest_xray\\' 

train_data = []
train_labels = []

for cond in ['\\NORMAL\\', '\\PNEUMONIA\\']:
    for img in (os.listdir(path + 'train' + cond)):
          img = cv2.imread(path+'train'+cond+img, cv2.IMREAD_COLOR)
          img = cv2.resize(img, (150, 150), cv2.INTER_AREA)
          img = img.astype('float32') / 255
          if cond=='\\NORMAL\\':
              label = 0
          elif cond=='\\PNEUMONIA\\':
              label = 1

          train_data.append(img) 
          train_labels.append(label)
        
train_data = np.array(train_data)
train_labels = np.array(train_labels)
train_data, train_labels = shuffle(train_data, train_labels)

### Здесь происходит загрузка данных для тестирование сети

In [0]:
test_data = []
test_labels = []

for cond in ['\\NORMAL\\', '\\PNEUMONIA\\']:
      for img in (os.listdir(path + 'test' + cond)):
          img = cv2.imread(path+'test'+cond+img, cv2.IMREAD_COLOR)
          img = cv2.resize(img, (150, 150), cv2.INTER_AREA)
          img = img.astype('float32') / 255
          if cond=='\\NORMAL\\':
              label = 0
          elif cond=='\\PNEUMONIA\\':
              label = 1
              
          test_data.append(img)
          test_labels.append(label)
        
test_data = np.array(test_data)
test_labels = np.array(test_labels)

### Как видно всего в датасете 5840 снимков. 5126 из них в обучающем наборе, 624 в тестирующем

In [0]:
print(len(test_data))
print(len(train_data))

624
5216


### Здесь находится функция для создания самой модели. Её архитектура немного напоминает VGG16, но она менее глубокая (так как датасет совсем небольшой, это позволяет уменьшить переобучаемость) и состоит из 5 convolution-блоков, начиная с 16х3х3 и затем увеличивая размерность с каждым разом в 2 раза. Затем после 5-и свёрточных блоков идёт полносвязный, состоящий из 3-ёх скрытых слоёв и одного выходного. В качестве активации у всех слоёв используется ReLU, у последнего — сигмоида.

In [0]:
def create_model():
  inputs = Input(shape=(150, 150, 3))

  # Первый блок свёртки
  x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
  x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = MaxPool2D(pool_size=(2, 2))(x)

  # Второй блок свёртки
  x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(pool_size=(2, 2))(x)

  # Третий блок свёртки
  x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(pool_size=(2, 2))(x)

  # Четвёртый блок свёртки
  x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(pool_size=(2, 2))(x)
  x = Dropout(rate=0.2)(x)

  # Пятый блок свёртки
  x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(pool_size=(2, 2))(x)
  x = Dropout(rate=0.2)(x)

  # Полносвязный слой
  x = Flatten()(x)
  x = Dense(units=512, activation='relu')(x)
  x = Dropout(rate=0.7)(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dropout(rate=0.5)(x)
  x = Dense(units=64, activation='relu')(x)
  x = Dropout(rate=0.3)(x)

  # Output layer
  output = Dense(units=1, activation='sigmoid')(x)

  # Creating model and compiling
  model = Model(inputs=inputs, outputs=output)
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

### Это же модель, которая суммирует предсказания 5-и моделей, со структорой выше. Она полностью полносвязная и состоит из 5-и входных нейронов, двух скрытых слоёв и одного выходного.

In [0]:
def create_top_model():
    #Simple fully-connected model for final decision
    inputs = Input(shape=(5, ))
    
    x = Dense(units=512, activation='relu')(inputs)
    x = Dense(units=256, activation='relu')(x)
    
    output = Dense(units=1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
checkpoint = ModelCheckpoint(filepath='top_model.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

### Так как обучение велось непосредственно в Colab, здесь находится лишь модуль с загрузкой всех пяти моделей 

In [0]:
models_path = '\\models\\'

models = [create_model() for i in range(5)]
for i in range(5):
  models[i].load_weights(os.getcwd() + models_path + 'best_weights-model' + str(i) + '.hdf5', verbose=0)

### Здесь находится функция для создания "слепка" из предсказаний моделей. Мы совмещаем предсказания 5-и моделей, для дальнейшего обучения полносвязной модели

In [0]:
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# Предсказания
		yhat = model.predict(inputX, verbose=0)
		# Стакаем предсказания [строки, модели, вероятности]
		if stackX is None:
			stackX = yhat
		else:
			stackX = np.dstack((stackX, yhat))
	# Уменьшаем размерность до [строки, модели х вероятности]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

### Здесь мы обучаем саму полносвязную модель, используя callback-и для понижения скорости обучения и сохранения весов лучшей модели

In [0]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

def fit_model(members, inputX, inputy):
	# Получаем стаканный датасет
	stackedX = stacked_dataset(members, inputX)
	# Обучаем модельку
	model = create_top_model()
	model.fit(stackedX, inputy, epochs=100, callbacks=[checkpoint, lr_reduce], validation_split=0.2)
	return model

model = fit_model(models, train_data, train_labels)

Train on 4172 samples, validate on 1044 samples
Epoch 1/100
4172/4172 [==============================] - 1s 153us/step - loss: 0.1586 - accuracy: 0.9331 - val_loss: 0.1035 - val_accuracy: 0.9655
Epoch 2/100
4172/4172 [==============================] - 1s 121us/step - loss: 0.0935 - accuracy: 0.9703 - val_loss: 0.1032 - val_accuracy: 0.9665
Epoch 3/100
4172/4172 [==============================] - 0s 115us/step - loss: 0.0933 - accuracy: 0.9703 - val_loss: 0.0966 - val_accuracy: 0.9674

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 4/100
4172/4172 [==============================] - 0s 114us/step - loss: 0.0895 - accuracy: 0.9710 - val_loss: 0.0955 - val_accuracy: 0.9674
Epoch 5/100
4172/4172 [==============================] - 0s 107us/step - loss: 0.0887 - accuracy: 0.9722 - val_loss: 0.0965 - val_accuracy: 0.9674

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.
Epoch 6/100
4172/4172 [==============================] 

4172/4172 [==============================] - 0s 105us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665
Epoch 45/100
4172/4172 [==============================] - 0s 117us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665

Epoch 00045: ReduceLROnPlateau reducing learning rate to 3.138105874196098e-15.
Epoch 46/100
4172/4172 [==============================] - 0s 118us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665
Epoch 47/100
4172/4172 [==============================] - 1s 120us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665

Epoch 00047: ReduceLROnPlateau reducing learning rate to 9.414317622588293e-16.
Epoch 48/100
4172/4172 [==============================] - 0s 117us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665
Epoch 49/100
4172/4172 [==============================] - 0s 118us/step - loss: 0.0872 - accuracy: 0.9732 - val


Epoch 00087: ReduceLROnPlateau reducing learning rate to 3.28257003487461e-26.
Epoch 88/100
4172/4172 [==============================] - 0s 116us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665
Epoch 89/100
4172/4172 [==============================] - 0s 115us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665

Epoch 00089: ReduceLROnPlateau reducing learning rate to 9.847710289513105e-27.
Epoch 90/100
4172/4172 [==============================] - 0s 118us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665
Epoch 91/100
4172/4172 [==============================] - 0s 108us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665

Epoch 00091: ReduceLROnPlateau reducing learning rate to 2.9543131330762504e-27.
Epoch 92/100
4172/4172 [==============================] - 0s 109us/step - loss: 0.0872 - accuracy: 0.9732 - val_loss: 0.0947 - val_accuracy: 0.9665
Epoch 93/100
4172

### Это функция для получения конечного предсказания. Модель получает предсказания 5-и других моделей и затем возвращает своё

In [0]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

### Измерение точности модели на наборе тестовых данных. Как видно, точность составляет почти 93%

In [0]:
yhat = stacked_prediction(models, model, test_data)
for i in range(len(yhat)):
    if yhat[i] > 0.95:
        yhat[i] = 1
    else:
        yhat[i] = 0
acc = accuracy_score(test_labels, yhat)
print('Stacked Test Accuracy:', (acc * 100))


Stacked Test Accuracy: 92.94871794871796


In [0]:
#Небольшой модуль для очистки видеопамяти. На всякий случай.
from numba import cuda
cuda.select_device(0)
cuda.close()